<a href="https://colab.research.google.com/github/Raysk1/machineLearning/blob/main/PROYECTO_02_FRANCO_GARCIA_OSCAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IMPORTANDO LIBRERIAS

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

#DESCRIBIENDO EL DATASET


In [2]:
#Cargando el dataset
df = pd.read_csv("/content/OnlineRetail.csv", sep=";")
#mostrando el dataset
display(df)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [3]:
#Mostrando informacion del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [4]:
#Mostrando el total de nulos por columna
print(df.isna().sum())

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


#LIMPIANDO EL DATASET

In [5]:
#eliminando valores nulos
df.dropna(inplace= True)
print(df.isna().sum())

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


In [6]:
#Calculando la media, mediana, maximo, minimo, etc
df.describe()

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [7]:
#Eliminando valores que no pueden ser posibles
df.drop(df.loc[df['Quantity'] < 0].index, inplace=True)
df.drop(df.loc[df['UnitPrice'] < 0].index, inplace=True)


In [8]:
#valores unicos por colunma
df.nunique()

InvoiceNo      18536
StockCode       3665
Description     3877
Quantity         302
InvoiceDate    17286
UnitPrice        441
CustomerID      4339
Country           37
dtype: int64

#CALCULANDO EL RFM

In [9]:
#Separando la fecha de la hora
df[['Date','Hour']] = df.InvoiceDate.str.split(' ', expand= True)
#Aplicando el tipo de dato a la columna
df['Date'] = pd.to_datetime(df['Date'])
#Eliminando columna InvoiceDate
df.drop(['InvoiceDate'], axis= 1)

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Hour
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010-12-01,8:26
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010-12-01,8:26
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,2010-12-01,8:26
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,2010-12-01,8:26
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,2010-12-01,8:26
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,0.85,12680.0,France,2011-12-09,12:50
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2.10,12680.0,France,2011-12-09,12:50
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,4.15,12680.0,France,2011-12-09,12:50
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,4.15,12680.0,France,2011-12-09,12:50


In [10]:
#Calculando recency
df_recency = df.groupby(by='CustomerID',
                        as_index=False)['Date'].max()
df_recency.columns = ['CustomerID', 'LastPurchaseDate']
recent_date = df_recency['LastPurchaseDate'].max()
df_recency['Recency'] = df_recency['LastPurchaseDate'].apply(
     lambda x: (recent_date - x).days)
display(df_recency)

,CustomerID,LastPurchaseDate,Recency
0,12346.0,2011-01-18,325
1,12347.0,2011-12-07,2
2,12348.0,2011-09-25,75
3,12349.0,2011-11-21,18
4,12350.0,2011-02-02,310
...,...,...,...
4334,18280.0,2011-03-07,277
4335,18281.0,2011-06-12,180
4336,18282.0,2011-12-02,7
4337,18283.0,2011-12-06,3


In [11]:
#Calculando frequency
df_frequency = df.drop_duplicates().groupby(
    by=['CustomerID'], as_index=False)['Date'].count()
df_frequency.columns = ['CustomerID', 'Frequency']
display(df_frequency)


,CustomerID,Frequency
0,12346.0,1
1,12347.0,182
2,12348.0,31
3,12349.0,73
4,12350.0,17
...,...,...
4334,18280.0,10
4335,18281.0,7
4336,18282.0,12
4337,18283.0,721


In [12]:
#Creando la columna Total
df['Total'] = df['Quantity']*df['UnitPrice']
display(df)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Date,Hour,Total
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,2010-12-01,8:26,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,2010-12-01,8:26,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,2010-12-01,8:26,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,2010-12-01,8:26,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,2010-12-01,8:26,20.34
...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,2011-12-09,12:50,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,2011-12-09,12:50,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,2011-12-09,12:50,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,2011-12-09,12:50,16.60


In [13]:
#calculando monetary value
df_monetary = df.groupby(by='CustomerID', as_index=False)['Total'].sum()
df_monetary.columns = ['CustomerID', 'Monetary']
display(df_monetary)

,CustomerID,Monetary
0,12346.0,77183.60
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40
...,...,...
4334,18280.0,180.60
4335,18281.0,80.82
4336,18282.0,178.05
4337,18283.0,2094.88


In [14]:
#Uniendo dfs
df_rfm = df_recency.merge(df_frequency, on='CustomerID')
df_rfm = df_rfm.merge(df_monetary, on='CustomerID').drop(
    columns='LastPurchaseDate')
display(df_rfm)

,CustomerID,Recency,Frequency,Monetary
0,12346.0,325,1,77183.60
1,12347.0,2,182,4310.00
2,12348.0,75,31,1797.24
3,12349.0,18,73,1757.55
4,12350.0,310,17,334.40
...,...,...,...,...
4334,18280.0,277,10,180.60
4335,18281.0,180,7,80.82
4336,18282.0,7,12,178.05
4337,18283.0,3,721,2094.88


In [15]:
df_rfm['R_rank'] = df_rfm['Recency'].rank(ascending=False)
df_rfm['F_rank'] = df_rfm['Frequency'].rank(ascending=True)
df_rfm['M_rank'] = df_rfm['Monetary'].rank(ascending=True)
 
# Normalizando el rango de los clientes
df_rfm['R_rank_norm'] = (df_rfm['R_rank']/df_rfm['R_rank'].max())*100
df_rfm['F_rank_norm'] = (df_rfm['F_rank']/df_rfm['F_rank'].max())*100
df_rfm['M_rank_norm'] = (df_rfm['F_rank']/df_rfm['M_rank'].max())*100
 
df_rfm.drop(columns=['R_rank', 'F_rank', 'M_rank'], inplace=True)
 
display(df_rfm)

,CustomerID,Recency,Frequency,Monetary,R_rank_norm,F_rank_norm,M_rank_norm
0,12346.0,325,1,77183.60,3.759833,0.841208,0.841208
1,12347.0,2,182,4310.00,96.170754,88.234616,88.234616
2,12348.0,75,31,1797.24,38.084220,42.359991,42.359991
3,12349.0,18,73,1757.55,74.259602,67.100714,67.100714
4,12350.0,310,17,334.40,5.263767,24.959668,24.959668
...,...,...,...,...,...,...,...
4334,18280.0,277,10,180.60,8.213790,14.346624,14.346624
4335,18281.0,180,7,80.82,19.898195,9.380041,9.380041
4336,18282.0,7,12,178.05,88.836187,17.722978,17.722978
4337,18283.0,3,721,2094.88,94.042110,99.297073,99.297073


In [16]:
#Calculando el Score

df_rfm['RFM_Score'] = 0.15*df_rfm['R_rank_norm']+0.28 * \
   df_rfm['F_rank_norm']+0.57*df_rfm['M_rank_norm']
df_rfm['RFM_Score'] *= 0.05
df_rfm = df_rfm.round(2)
df_rfm[['CustomerID', 'RFM_Score']].head(7)

,CustomerID,RFM_Score
0,12346.0,0.06
1,12347.0,4.47
2,12348.0,2.09
3,12349.0,3.41
4,12350.0,1.10
5,12352.0,3.46
6,12353.0,0.33


* puntuación rfm >4.5 : Mejor cliente
* 4.5 > puntuación rfm > 4 : Cliente de alto valor
* 4>puntuación rfm >3: Cliente de valor medio
* 3>puntuación rfm>1.6: cliente de bajo valor
* Puntuación rfm <1.6: Cliente perdido

In [17]:

df_rfm["Customer_segment"] = np.where(df_rfm['RFM_Score'] >
                                      4.5, "Top Customer",
                                      (np.where(
                                        df_rfm['RFM_Score'] > 4,
                                        "High value Customer",
                                        (np.where(
    df_rfm['RFM_Score'] > 3,
                             "Medium Value Customer",
                             np.where(df_rfm['RFM_Score'] > 1.6,
                            'Low Value Customer', 'Lost Customer'))))))
df_rfm[['CustomerID', 'RFM_Score', 'Customer_segment']].head(20)

,CustomerID,RFM_Score,Customer_segment
0,12346.0,0.06,Lost Customer
1,12347.0,4.47,High value Customer
2,12348.0,2.09,Low Value Customer
3,12349.0,3.41,Medium Value Customer
4,12350.0,1.10,Lost Customer
5,12352.0,3.46,Medium Value Customer
6,12353.0,0.33,Lost Customer
7,12354.0,2.67,Low Value Customer
8,12355.0,0.93,Lost Customer
9,12356.0,3.11,Medium Value Customer
